# Reconstitution INPI

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
path = r'C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\programme_matching\data\output'
list_issue = []
df_matched = pd.DataFrame()
for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
    for name in tqdm(files):
        if name.endswith((".gz")):
            path_gz = '{}\{}'.format(root, name)
            
            df_matched = df_matched.append(
            pd.read_csv(path_gz, compression = 'gzip',low_memory = False))

In [ ]:
path_inpi = r'C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\programme_matching\data\input\inpi_etb_2_4979272.gz'
df_inpi = pd.read_csv(path_inpi,compression = 'gzip',low_memory = False)

In [ ]:
df_inpi.shape[0] - df_matched.shape[0]

In [ ]:
df_matched_full = df_inpi.merge(df_matched[['index', 'siret', 'ncc', 'Adress_new']], on = 'index',how = 'inner')

In [ ]:
df_matched_full.to_csv('data\\inpi_etb_test_{}.gz'.format(
    df_matched_full.shape[0]),
                       compression='gzip', index = False) 

In [ ]:
df_matched_full.head(2)

# test app

# Save to SQL

In [ ]:
list(df_)

In [ ]:
from sqlalchemy import create_engine
import sqlite3
conn = sqlite3.connect('siren_inpi.db')
c = conn.cursor()

In [ ]:
c.execute("CREATE TABLE SIREN (Code Greffe,Nom_Greffe,Numero_Gestion,siren,Type, \
 Siège_PM,RCS_Registre,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,  \
 Code_Postal, Ville,Code_Commune,Pays,Domiciliataire_Nom,Domiciliataire_Siren,  \
 Domiciliataire_Greffe,Domiciliataire_Complément,Siege_Domicile_Représentant,Nom_Commercial,  \
 Enseigne,Activité_Ambulante, Activité_Saisonnière,Activité_Non_Sédentaire,  \
 Date_Début_Activité,Activité, Origine_Fonds,Origine_Fonds_Info,  \
 Type_Exploitation,ID_Etablissement,Date_Greffe,Libelle_Evt, count, siret, ncc, Adress_new)") 
conn.commit()

df_.drop(columns = 'index').compute().to_sql('SIREN', conn, if_exists='replace', index = False)

In [ ]:
test = df_['siren'].compute().drop_duplicates()#.to_csv(r'siren.csv', index = False)

In [ ]:
test.to_csv(r'data\siren.csv', index = False)

# Test APP

In [1]:
import pandas as pd

In [2]:
list_siren = pd.read_csv(r'data\siren.csv')
list_siren.head()

,siren
0,813543063
1,800897092
2,440432052
3,534728381
4,750647851


# Archive

In [ ]:
import json, os, re
from dask.diagnostics import ProgressBar
from dask.multiprocessing import get
import dask.dataframe as dd
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
pbar = ProgressBar()
pbar.register()


In [ ]:
def on_change(change):
    #

    #"df = pd.DataFrame(c.fetchall())  
    #output_brand.clear_output()
    #with output_brand:
    conn = sqlite3.connect('siren_inpi.db')
    c = conn.cursor()
    if change['type'] == 'change' and change['name'] == 'value':
            query = '''SELECT * FROM SIREN WHERE siren = "%s"''' % change['new']
        
            c.execute(query)

            df = pd.DataFrame(c.fetchall(), columns= cols)
        
            display(df)

text_siren.observe(on_change)
display(text_siren)

In [ ]:
text_siren = widgets.Text(
    value='813543063',
    placeholder='Paste ticket description here!',
    description='String:',
    disabled=False
)
cols = ["Code" "Greffe","Nom_Greffe","Numero_Gestion","siren","Type", \
 "Siège_PM","RCS_Registre","Adresse_Ligne1","Adresse_Ligne2","Adresse_Ligne3",  \
 "Code_Postal", "Ville","Code_Commune","Pays","Domiciliataire_Nom","Domiciliataire_Siren",  \
 "Domiciliataire_Greffe","Domiciliataire_Complément","Siege_Domicile_Représentant","Nom_Commercial",  \
 "Enseigne","Activité_Ambulante", "Activité_Saisonnière","Activité_Non_Sédentaire",  \
 "Date_Début_Activité","Activité", "Origine_Fonds","Origine_Fonds_Info",  \
 "Type_Exploitation","ID_Etablissement","Date_Greffe","Libelle_Evt", "count", "siret", "ncc", "Adress_new"]

In [ ]:
df_ = pd.read_csv('data\\inpi_etb_test_4911141.gz',
        compression='gzip',dtype={'Code_Commune': 'object',
       'Code_Postal': 'object',
       'Domiciliataire_Greffe': 'object'}, low_memory=False)

In [ ]:
def grouptolist(df):
    keys, values = df.sort_values('siren').values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    df2 = pd.DataFrame({'siren': ukeys, 'ncc': [list(a) for a in arrays]})
    return df2

In [ ]:
df_ = dd.read_csv('data\\inpi_etb_test_4911141.gz',
        blocksize=None,compression='gzip',dtype={'Code_Commune': 'object',
       'Code_Postal': 'object',
       'Domiciliataire_Greffe': 'object'})

In [ ]:
%time 
test = grouptolist(df_[['siren', 'ncc']].drop_duplicates().compute())

In [ ]:
%time 
test = grouptolist(df_[['siren', 'ncc']].drop_duplicates())

In [ ]:
df_json = test.to_json(orient='columns')

In [ ]:
#test.to_csv('insee_inpi.csv', index = False)

In [ ]:
with open('insee_inpi.json', 'w') as outfile:
    json.dump(df_json, outfile)

In [ ]:
all_options = {
    'America': ['New York City', 'San Francisco', 'Cincinnati'],
    'Canada': [u'Montréal', 'Toronto', 'Ottawa']
}
[{'label': k, 'value': k} for k in all_options.keys()]

In [ ]:
[{'label': i, 'value': i} for i in all_options['America']]

In [ ]:
df_json['ncc'] = df_json['ncc'].str.replace("'", "")

In [ ]:
df_json.head()

In [ ]:
dic_ = []
for city in test[test['siren'].isin([160020012])]['ncc']:
    print(city)
    for c in city:
        dic_1 = {'label': c, 'value': c}
        dic_.append(dic_1)
dic_

In [ ]:
dic_ = []
df_json = pd.read_csv(r'App\insee_inpi.csv'
                      )
for city in df_json[df_json['siren'].isin([160020012])]['ncc']:
    print(city)
    for c in city:
        dic_1 = {'label': c, 'value': c}
        dic_.append(dic_1)
dic_ 

In [ ]:
[{'label': i, 'value': i} for i in test[test['siren'].isin([160020012])]['ncc']]

In [ ]:
df_json[df_json['siren'].isin(['130008857'])]['ncc']

In [ ]:
list_siren = df_['siren'].drop_duplicates().to_list()

In [ ]:
test = df_[['siren', 'ncc']].drop_duplicates().agg({'b':lambda x: list(x)})

In [ ]:
test = df_[['siren', 'Adress_new']].drop_duplicates().groupby('siren')['Adress_new'].apply(list)

In [ ]:
dic_full = {}
from tqdm import tqdm
for x in tqdm(list_siren):
    add = df_.loc[
        df_['siren'].isin([x])]['Adress_new']
    
    dic_full[x] = add
dic_full